Bibliography:

* [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction](http://people.idsia.ch/~ciresan/data/icann2011.pdf)
    

In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


In [2]:
import cae

In [3]:
from helpers import *
from helper_modules import *
from multi_res_cae import *

In [4]:
%load_ext autoreload
%autoreload

In [5]:
%aimport helpers, helper_modules, multi_res_cae

In [6]:
%aimport

Modules to reload:
helper_modules helpers multi_res_cae

Modules to skip:



In [7]:
# frcae = MultiFullCAE((640,480))

In [8]:
# mrcae = MultiResCAE([640,480])

In [9]:
# Hyper Parameters
# num_epochs = 5
# batch_size = 100
# learning_rate = 0.001

num_epochs = 20
batch_size = 128
learning_rate = 0.0001

In [10]:
%%time

model = MultiFullCAE(in_img_shape=(32,32), channels=1, full_image_resize=(24,24)).cuda()
# model = MultiResCAE(in_img_shape=[32,32], channels=3, conv_layer_feat=[16, 32, 64],
#                  res_px=[[24, 24], [16, 16], [12, 12]], crop_sizes=[[32, 32], [24,24], [12, 12]],
#                  # conv_sizes = [(3,5,7), (3,5,7,11), (3,5,7,11)]  # this is too much I think
#                  # conv_sizes=[[1, 3, 5], [1, 3, 5], [1, 3, 5, 7]]  # test b
#                  conv_sizes=[[5, 7, 11], [3, 5, 7, 9], [1, 3, 5]]  # test c
#         ).cuda()

CPU times: user 1.5 s, sys: 519 ms, total: 2.01 s
Wall time: 2.02 s


In [11]:
# model.parameters

In [12]:
%%time
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

CPU times: user 755 µs, sys: 301 µs, total: 1.06 ms
Wall time: 1.07 ms


In [13]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 32, 32)
    return x


In [14]:
%%time

transformation = monochrome_preprocess(32,32)
#transformation = fullimage_preprocess(32,32)
#train_loader, test_loader = get_loaders(batch_size, transformation, dataset=datasets.CocoDetection)
train_loader, test_loader = get_loaders(batch_size, transformation)

Files already downloaded and verified
CPU times: user 712 ms, sys: 220 ms, total: 932 ms
Wall time: 931 ms


In [15]:
%%time

for epoch in range(num_epochs):
    for i, (img, labels) in enumerate(train_loader):
        img = Variable(img).cuda()
        # ===================forward=====================
#         print("encoding batch of  images")
        output = model(img)
#         print("computing loss")
        loss = criterion(output, img)
        # ===================backward====================
#         print("Backward ")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data))
    if epoch % 4 == 0:
        pic = to_img(output.cpu().data)
        in_pic = to_img(img.cpu().data)
        save_image(pic, './fmrcae_results/c_MergingLayer_in-32x32_3-5-7-11_out_image_{}.png'.format(epoch))
        save_image(in_pic, './fmrcae_results/c_MergingLayer_in-3-5-7-11_in_image_{}.png'.format(epoch))
#     if loss.data[0] < 0.21: #arbitrary number because I saw that it works well enough
#         break

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/leo/venv3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/leo/venv3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/leo/venv3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/leo/venv3/lib/python3.8/site-packages/torchvision/datasets/cifar.py", line 120, in __getitem__
    img = self.transform(img)
  File "/home/leo/venv3/lib/python3.8/site-packages/torchvision/transforms/transforms.py", line 67, in __call__
    img = t(img)
  File "/home/leo/venv3/lib/python3.8/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(

In [16]:
#torch.save("fmrcae_in-64x64_32x32_3-5-7-11.pth", model)
#torch.save("mrcae_in-32x32_.pth", model)